In [1]:
import numpy as np
import scipy as sp
import matplotlib as plt
import seaborn as sns
import pandas as pd

In [2]:
import nltk
from textblob import TextBlob

In [3]:
full_data = pd.read_csv('aita_clean.csv')

In [4]:
full_data.head()

,id,timestamp,title,body,edited,verdict,score,num_comments,is_asshole
0,arfcfl,1550365421,AITA - Getting Hugged At The Bar,\n\nThis all happened less than an hour ago.\...,False,not the asshole,3,14,0
1,cg8mxn,1563769917,AITA if i don't want to pay my friend 5 dollar...,"So, my friend bought herself, our other friend...",False,asshole,11,42,1
2,bajsje,1554663842,AITA For going to 'expensive' restaurants in s...,It's in Seattle if that matters. It seems peop...,1554686830.0,asshole,657,397,1
3,cq6voc,1565771883,"AITA for wanting ""free gas"", due to an attenda...",I was with my dad and driving the family car w...,1565773989.0,not the asshole,49,40,0
4,bz4m2k,1560207411,AITA I asked my dad to replace the fuel that h...,Am I the asshole for asking my dad to replace ...,False,not the asshole,4,10,0


In [5]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
id              10000 non-null object
timestamp       10000 non-null int64
title           10000 non-null object
body            9993 non-null object
edited          10000 non-null object
verdict         10000 non-null object
score           10000 non-null int64
num_comments    10000 non-null int64
is_asshole      10000 non-null int64
dtypes: int64(4), object(5)
memory usage: 703.2+ KB


In [6]:
empty_body = full_data[full_data['body'].isnull()]

In [7]:
empty_body

,id,timestamp,title,body,edited,verdict,score,num_comments,is_asshole
525,8m1373,1527248555,"AITA? Whenever I get into a religious debate, ...",NaN,False,asshole,6,24,1
3205,8bdpc6,1523413473,AITA for how I parked?,NaN,False,asshole,357,268,1
5334,93dym6,1533039397,AITA if i laugh when babies fall down and cry?...,NaN,False,not the asshole,12,29,0
5465,5lgkak,1483297705,AITA for thinking it's better to give money to...,NaN,False,not the asshole,20,5,0
7488,5hqo5f,1481471842,AITA for taking this picture and posting it to...,NaN,False,asshole,81,86,1
8012,azlo3l,1552258622,AITA after being offered a cigarette from a cl...,NaN,False,asshole,5,47,1
8451,3d6n5l,1436828279,AITA for being a dick to Amazon customer servi...,NaN,False,asshole,8,26,1


In [8]:
full_data['verdict'].value_counts()

not the asshole     6119
asshole             1988
no assholes here    1288
everyone sucks       605
Name: verdict, dtype: int64

In [9]:
full_data = full_data.dropna()

In [146]:
nta = full_data[full_data['verdict'] == 'not the asshole']
ah = full_data[full_data['verdict'] == 'asshole']

In [54]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9993 entries, 0 to 9999
Data columns (total 9 columns):
id              9993 non-null object
timestamp       9993 non-null int64
title           9993 non-null object
body            9993 non-null object
edited          9993 non-null object
verdict         9993 non-null object
score           9993 non-null int64
num_comments    9993 non-null int64
is_asshole      9993 non-null int64
dtypes: int64(4), object(5)
memory usage: 780.7+ KB


In [55]:
nta_ds = nta.sample(len(ah)) #downsample to match ah

In [56]:
nta_ds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1983 entries, 1053 to 2277
Data columns (total 9 columns):
id              1983 non-null object
timestamp       1983 non-null int64
title           1983 non-null object
body            1983 non-null object
edited          1983 non-null object
verdict         1983 non-null object
score           1983 non-null int64
num_comments    1983 non-null int64
is_asshole      1983 non-null int64
dtypes: int64(4), object(5)
memory usage: 154.9+ KB


In [57]:
ah.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1983 entries, 1 to 9989
Data columns (total 9 columns):
id              1983 non-null object
timestamp       1983 non-null int64
title           1983 non-null object
body            1983 non-null object
edited          1983 non-null object
verdict         1983 non-null object
score           1983 non-null int64
num_comments    1983 non-null int64
is_asshole      1983 non-null int64
dtypes: int64(4), object(5)
memory usage: 154.9+ KB


In [86]:
ah

,id,timestamp,title,body,edited,verdict,score,num_comments,is_asshole
1,cg8mxn,1563769917,AITA if i don't want to pay my friend 5 dollar...,"So, my friend bought herself, our other friend...",False,asshole,11,42,1
2,bajsje,1554663842,AITA For going to 'expensive' restaurants in s...,It's in Seattle if that matters. It seems peop...,1554686830.0,asshole,657,397,1
11,4vlby6,1470036194,AITA for throwing a couple of K-Pop Photocards...,"Ok, so last Saturday, BTS, a K-Pop group had a...",False,asshole,10,2,1
15,as9fwl,1550575964,AITA for making my girlfriend cancel her birth...,Next Saturday is my Birthday and my girlfriend...,False,asshole,17,27,1
17,dbonzb,1569907647,WIBTA if I found a new weight training partner?,I'm on a sports team (at a D1 NCAA school) and...,False,asshole,3,10,1
...,...,...,...,...,...,...,...,...,...
9970,b5xbjr,1553644895,AITA for telling a girl im seeing i didnt like...,Ok so title sounds striaght up but doesnt give...,1553661780.0,asshole,8,31,1
9972,btrhl0,1558991748,WIBTA if I reported my neighbors for overcrowd...,"They fill up the streets with their cars, some...",False,asshole,3,8,1
9973,6eapd8,1496183189,AITA for calling out a friend who says their i...,I have friends who keep referring to their chi...,False,asshole,54,31,1
9981,bweauk,1559587543,AITA For sleeping in my nieces bedroom because...,I was at my sister’s house (Family reasons- pl...,False,asshole,20,43,1


In [149]:
cln_data = pd.concat([ah,nta],0)
cln_data

,id,timestamp,title,body,edited,verdict,score,num_comments,is_asshole
1,cg8mxn,1563769917,AITA if i don't want to pay my friend 5 dollar...,"So, my friend bought herself, our other friend...",False,asshole,11,42,1
2,bajsje,1554663842,AITA For going to 'expensive' restaurants in s...,It's in Seattle if that matters. It seems peop...,1554686830.0,asshole,657,397,1
11,4vlby6,1470036194,AITA for throwing a couple of K-Pop Photocards...,"Ok, so last Saturday, BTS, a K-Pop group had a...",False,asshole,10,2,1
15,as9fwl,1550575964,AITA for making my girlfriend cancel her birth...,Next Saturday is my Birthday and my girlfriend...,False,asshole,17,27,1
17,dbonzb,1569907647,WIBTA if I found a new weight training partner?,I'm on a sports team (at a D1 NCAA school) and...,False,asshole,3,10,1
...,...,...,...,...,...,...,...,...,...
9994,cixb3f,1564328490,AITA for not letting my friend come over to my...,So friday evening around 6PM I decided to have...,False,not the asshole,28,15,0
9995,bt7i7h,1558874703,AITA: for hooking up with my ex while seeing s...,A little bit of backstory: my ex and I dated f...,False,not the asshole,7,7,0
9996,ddsioe,1570305985,AITA for kicking my mother in law out after sh...,Just like the title says. My mother in law cam...,1570306643.0,not the asshole,423,218,0
9997,awgmrm,1551523851,AITA for not attending the final ride out for ...,"I am an avid biker, don't drive a car, never h...",False,not the asshole,13,6,0


In [157]:
df = pd.DataFrame()
df['tblob'] = cln_data['title'].apply(lambda row: TextBlob(row))
df['pblob'] = cln_data['body'].apply(lambda row: TextBlob(row))
df

,tblob,pblob
1,"(A, I, T, A, , i, f, , i, , d, o, n, ', t, ...","(S, o, ,, , m, y, , f, r, i, e, n, d, , b, ..."
2,"(A, I, T, A, , F, o, r, , g, o, i, n, g, , ...","(I, t, ', s, , i, n, , S, e, a, t, t, l, e, ..."
11,"(A, I, T, A, , f, o, r, , t, h, r, o, w, i, ...","(O, k, ,, , s, o, , l, a, s, t, , S, a, t, ..."
15,"(A, I, T, A, , f, o, r, , m, a, k, i, n, g, ...","(N, e, x, t, , S, a, t, u, r, d, a, y, , i, ..."
17,"(W, I, B, T, A, , i, f, , I, , f, o, u, n, ...","(I, ', m, , o, n, , a, , s, p, o, r, t, s, ..."
...,...,...
9994,"(A, I, T, A, , f, o, r, , n, o, t, , l, e, ...","(S, o, , f, r, i, d, a, y, , e, v, e, n, i, ..."
9995,"(A, I, T, A, :, , f, o, r, , h, o, o, k, i, ...","(A, , l, i, t, t, l, e, , b, i, t, , o, f, ..."
9996,"(A, I, T, A, , f, o, r, , k, i, c, k, i, n, ...","(J, u, s, t, , l, i, k, e, , t, h, e, , t, ..."
9997,"(A, I, T, A, , f, o, r, , n, o, t, , a, t, ...","(I, , a, m, , a, n, , a, v, i, d, , b, i, ..."


In [161]:
df['pwords'] = df['pblob'].apply(lambda row: row.words)
df['ptags'] = df['pblob'].apply(lambda row: row.tags)
df['pbigram'] = df['pblob'].apply(lambda row: row.ngrams(n=2))
df['ppolarity'] = df['pblob'].apply(lambda row: row.sentiment.polarity)
df['pobjectivity'] = df['pblob'].apply(lambda row: row.sentiment.objectivity)


In [162]:
df['class'] = cln_data['is_asshole']

In [163]:
df

,tblob,pblob,pwords,twords,ptags,class
1,"(A, I, T, A, , i, f, , i, , d, o, n, ', t, ...","(S, o, ,, , m, y, , f, r, i, e, n, d, , b, ...","[So, my, friend, bought, herself, our, other, ...","[AITA, if, i, do, n't, want, to, pay, my, frie...","[(So, RB), (my, PRP$), (friend, NN), (bought, ...",1
2,"(A, I, T, A, , F, o, r, , g, o, i, n, g, , ...","(I, t, ', s, , i, n, , S, e, a, t, t, l, e, ...","[It, 's, in, Seattle, if, that, matters, It, s...","[AITA, For, going, to, 'expensive, restaurants...","[(It, PRP), ('s, VBZ), (in, IN), (Seattle, NNP...",1
11,"(A, I, T, A, , f, o, r, , t, h, r, o, w, i, ...","(O, k, ,, , s, o, , l, a, s, t, , S, a, t, ...","[Ok, so, last, Saturday, BTS, a, K-Pop, group,...","[AITA, for, throwing, a, couple, of, K-Pop, Ph...","[(Ok, NNP), (so, IN), (last, JJ), (Saturday, N...",1
15,"(A, I, T, A, , f, o, r, , m, a, k, i, n, g, ...","(N, e, x, t, , S, a, t, u, r, d, a, y, , i, ...","[Next, Saturday, is, my, Birthday, and, my, gi...","[AITA, for, making, my, girlfriend, cancel, he...","[(Next, JJ), (Saturday, NNP), (is, VBZ), (my, ...",1
17,"(W, I, B, T, A, , i, f, , I, , f, o, u, n, ...","(I, ', m, , o, n, , a, , s, p, o, r, t, s, ...","[I, 'm, on, a, sports, team, at, a, D1, NCAA, ...","[WIBTA, if, I, found, a, new, weight, training...","[(I, PRP), ('m, VBP), (on, IN), (a, DT), (spor...",1
...,...,...,...,...,...,...
9994,"(A, I, T, A, , f, o, r, , n, o, t, , l, e, ...","(S, o, , f, r, i, d, a, y, , e, v, e, n, i, ...","[So, friday, evening, around, 6PM, I, decided,...","[AITA, for, not, letting, my, friend, come, ov...","[(So, RB), (friday, JJ), (evening, VBG), (arou...",0
9995,"(A, I, T, A, :, , f, o, r, , h, o, o, k, i, ...","(A, , l, i, t, t, l, e, , b, i, t, , o, f, ...","[A, little, bit, of, backstory, my, ex, and, I...","[AITA, for, hooking, up, with, my, ex, while, ...","[(A, DT), (little, JJ), (bit, NN), (of, IN), (...",0
9996,"(A, I, T, A, , f, o, r, , k, i, c, k, i, n, ...","(J, u, s, t, , l, i, k, e, , t, h, e, , t, ...","[Just, like, the, title, says, My, mother, in,...","[AITA, for, kicking, my, mother, in, law, out,...","[(Just, RB), (like, IN), (the, DT), (title, NN...",0
9997,"(A, I, T, A, , f, o, r, , n, o, t, , a, t, ...","(I, , a, m, , a, n, , a, v, i, d, , b, i, ...","[I, am, an, avid, biker, do, n't, drive, a, ca...","[AITA, for, not, attending, the, final, ride, ...","[(I, PRP), (am, VBP), (an, DT), (avid, NN), (b...",0


In [43]:
blob.tokens

WordList(['So', ',', 'my', 'friend', 'bought', 'herself', ',', 'our', 'other', 'friend', 'and', 'I', 'a', 'pizza', 'to', 'eat', 'for', 'lunch', '.', 'Me', 'and', 'other', 'friend', 'ate', '1', 'slice', 'of', 'pizza', 'from', 'an', 'extra', 'large', 'pizza', '.', 'Other', 'friend', 'has', 'already', 'paid', 'my', 'friend', 'that', 'bought', 'the', 'pizza', '5', 'dollars', '.', 'I', 'am', 'trying', 'to', 'save', 'money', 'wherever', 'i', 'can', ',', 'but', 'she', 'really', 'wants', 'me', 'to', 'pay', 'her', '5', 'dollars', '``', 'so', 'its', 'fair', "''", '.', 'AITA', '?'])